# PaBiRoboy dynamic equations

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the reference frames.

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify, Matrix
from sympy import trigsimp
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, Particle, inertia, RigidBody, KanesMethod
from numpy import deg2rad, rad2deg, array, zeros, linspace
from sympy.physics.vector import init_vprinting, vlatex
import numpy as np
from scipy.integrate import odeint
from sympy.utilities.codegen import codegen
from pydy.codegen.ode_function_generators import generate_ode_function
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
rcParams['figure.figsize'] = (14.0, 6.0)

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

## Interial frames

In [3]:
inertial_frame = ReferenceFrame('I')
lower_leg_left_frame = ReferenceFrame('R_1')
upper_leg_left_frame = ReferenceFrame('R_2')
hip_frame = ReferenceFrame('R_3')
upper_leg_right_frame = ReferenceFrame('R_4')
lower_leg_right_frame = ReferenceFrame('R_5')

## Angles

In [4]:
theta0, theta1, theta2, theta3, phi = dynamicsymbols('theta0, theta1, theta2, theta3, phi')

In [5]:
lower_leg_left_frame.orient(inertial_frame, 'Axis', (phi, inertial_frame.z))
simplify(lower_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(phi), sin(phi), 0],
[-sin(phi), cos(phi), 0],
[        0,        0, 1]])

In [6]:
upper_leg_left_frame.orient(lower_leg_left_frame, 'Axis', (theta0, -lower_leg_left_frame.z))
simplify(upper_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(phi - theta0), sin(phi - theta0), 0],
[-sin(phi - theta0), cos(phi - theta0), 0],
[                 0,                 0, 1]])

In [7]:
hip_frame.orient(upper_leg_left_frame, 'Axis', (theta1, -upper_leg_left_frame.z))
hip_frame.dcm(inertial_frame)

Matrix([
[(-sin(theta0)*sin(theta1) + cos(theta0)*cos(theta1))*cos(phi) - (-sin(theta0)*cos(theta1) - sin(theta1)*cos(theta0))*sin(phi), (-sin(theta0)*sin(theta1) + cos(theta0)*cos(theta1))*sin(phi) + (-sin(theta0)*cos(theta1) - sin(theta1)*cos(theta0))*cos(phi), 0],
[-(-sin(theta0)*sin(theta1) + cos(theta0)*cos(theta1))*sin(phi) + (sin(theta0)*cos(theta1) + sin(theta1)*cos(theta0))*cos(phi),  (-sin(theta0)*sin(theta1) + cos(theta0)*cos(theta1))*cos(phi) + (sin(theta0)*cos(theta1) + sin(theta1)*cos(theta0))*sin(phi), 0],
[                                                                                                                            0,                                                                                                                             0, 1]])

In [8]:
upper_leg_right_frame.orient(hip_frame, 'Axis', (theta2, -hip_frame.z))
simplify(upper_leg_right_frame.dcm(inertial_frame))

Matrix([
[cos(-phi + theta0 + theta1 + theta2), -sin(-phi + theta0 + theta1 + theta2), 0],
[sin(-phi + theta0 + theta1 + theta2),  cos(-phi + theta0 + theta1 + theta2), 0],
[                                   0,                                     0, 1]])

In [9]:
lower_leg_right_frame.orient(upper_leg_right_frame, 'Axis', (theta3, -upper_leg_right_frame.z))
simplify(lower_leg_right_frame.dcm(inertial_frame))

Matrix([
[cos(-phi + theta0 + theta1 + theta2 + theta3), -sin(-phi + theta0 + theta1 + theta2 + theta3), 0],
[sin(-phi + theta0 + theta1 + theta2 + theta3),  cos(-phi + theta0 + theta1 + theta2 + theta3), 0],
[                                            0,                                              0, 1]])

## Points and Locations

In [10]:
origin = Point('Origin')
ankle_left = Point('AnkleLeft')
knee_left = Point('KneeLeft')
hip_left = Point('HipLeft')
hip_center = Point('HipCenter')
hip_right = Point('HipRight')
knee_right = Point('KneeRight')
ankle_right = Point('AnkleRight')

In [11]:
lower_leg_length, upper_leg_length, hip_length = symbols('l1, l2, l3')

In [12]:
origin = Point('Origin')
ankle_left = Point('AnkleLeft')
knee_left = Point('KneeLeft')
hip_left = Point('HipLeft')
hip_center = Point('HipCenter')
hip_right = Point('HipRight')
knee_right = Point('KneeRight')
ankle_right = Point('AnkleRight')

# here go the lengths of your robots links
lower_leg_length, upper_leg_length, hip_length = symbols('l1, l2, l3')

ankle_left.set_pos(origin, (0 * inertial_frame.y)+(0 * inertial_frame.x))
#ankle_left.pos_from(origin).express(inertial_frame).simplify()

knee_left.set_pos(ankle_left, lower_leg_length * lower_leg_left_frame.y)
#knee_left.pos_from(origin).express(inertial_frame).simplify()

hip_left.set_pos(knee_left, upper_leg_length * upper_leg_left_frame.y)
#hip_left.pos_from(origin).express(inertial_frame).simplify()

hip_center.set_pos(hip_left, hip_length/2 * hip_frame.x)
#hip_center.pos_from(origin).express(inertial_frame).simplify()

hip_right.set_pos(hip_center, hip_length/2 * hip_frame.x)
#hip_right.pos_from(origin).express(inertial_frame).simplify()

knee_right.set_pos(hip_right, upper_leg_length * -upper_leg_right_frame.y)
#knee_right.pos_from(origin).express(inertial_frame).simplify()

ankle_right.set_pos(knee_right, lower_leg_length * -lower_leg_right_frame.y)
#ankle_right.pos_from(origin).express(inertial_frame).simplify()


In [13]:
lower_leg_left_com_length = lower_leg_length/2
upper_leg_left_com_length = upper_leg_length/2
hip_com_length = hip_length/2
upper_leg_right_com_length = upper_leg_length/2
lower_leg_right_com_length = lower_leg_length/2

lower_leg_left_mass_center = Point('L_COMleft')
upper_leg_left_mass_center = Point('U_COMleft')
hip_mass_center = Point('H_COMleft')
upper_leg_right_mass_center = Point('U_COMright')
lower_leg_right_mass_center = Point('L_COMright')

In [14]:
lower_leg_left_mass_center.set_pos(ankle_left, lower_leg_left_com_length * lower_leg_left_frame.y)
#lower_leg_left_mass_center.pos_from(origin).express(inertial_frame).simplify()

upper_leg_left_mass_center.set_pos(knee_left, upper_leg_left_com_length * upper_leg_left_frame.y)
#upper_leg_left_mass_center.pos_from(origin).express(inertial_frame).simplify()

hip_mass_center.set_pos(hip_center, 0 * hip_frame.x)
#hip_mass_center.pos_from(origin).express(inertial_frame).simplify()

upper_leg_right_mass_center.set_pos(knee_right, upper_leg_right_com_length * upper_leg_right_frame.y)
#upper_leg_right_mass_center.pos_from(origin).express(inertial_frame).simplify()

lower_leg_right_mass_center.set_pos(ankle_right, lower_leg_right_com_length * lower_leg_right_frame.y)
lower_leg_right_mass_center.pos_from(origin).express(inertial_frame).simplify()

(-l1*sin(-phi + theta0 + theta1 + theta2 + theta3)/2 - l1*sin(phi) - l2*sin(phi - theta0) - l2*sin(-phi + theta0 + theta1 + theta2) + l3*cos(-phi + theta0 + theta1))*I.x + (-l1*cos(-phi + theta0 + theta1 + theta2 + theta3)/2 + l1*cos(phi) + l2*cos(phi - theta0) - l2*cos(-phi + theta0 + theta1 + theta2) - l3*sin(-phi + theta0 + theta1))*I.y

## Kinematical Differential Equations

In [15]:
omega0, omega1, omega2, omega3, psi = dynamicsymbols('omega0, omega1, omega2, omega3, psi')
kinematical_differential_equations = [omega0 - theta0.diff(),
                                      omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff(),
                                        psi - phi.diff(),                                
                                        ]

kinematical_differential_equations

[omega0 - theta0', omega1 - theta1', omega2 - theta2', omega3 - theta3', psi - phi']

## Angular Velocities

In [16]:
lower_leg_left_frame.set_ang_vel(inertial_frame,         0 * inertial_frame.z)
#lower_leg_left_frame.ang_vel_in(inertial_frame)

upper_leg_left_frame.set_ang_vel(upper_leg_left_frame,   omega0     * inertial_frame.z)
#upper_leg_left_frame.ang_vel_in(inertial_frame)

hip_frame.set_ang_vel(hip_frame,   omega1     * inertial_frame.z)
#hip_frame.ang_vel_in(inertial_frame)

upper_leg_right_frame.set_ang_vel(upper_leg_right_frame, omega2     * inertial_frame.z)
#upper_leg_right_frame.ang_vel_in(inertial_frame)

lower_leg_right_frame.set_ang_vel(lower_leg_right_frame, omega3     * inertial_frame.z)
lower_leg_right_frame.ang_vel_in(inertial_frame)


- theta3'*R_4.z - theta2'*R_3.z - theta1'*R_2.z - theta0'*R_1.z

# Linear Velocities

Finally, the linear velocities of the mass centers are needed. Starting at the ankle_left which has a velocity of 0.

In [17]:
ankle_left.set_vel(inertial_frame, 0)
origin.set_vel(inertial_frame, 0)

Working our way up the chain we can make use of the fact that the joint points are located on two rigid bodies. Any fixed point in a reference frame can be computed if the linear velocity of another point on that frame is known and the frame's angular velocity is known.

$$^I\mathbf{v}^{P_2} = ^I\mathbf{v}^{P_1} + ^I\omega^A \times \mathbf{r}^{\frac{P_2}{P_1}}$$

The `Point.v2pt_theory()` method makes it easy to do this calculation.

In [18]:
knee_left.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
#knee_left.vel(inertial_frame)

hip_left.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
#hip_left.vel(inertial_frame)

hip_center.v2pt_theory(hip_left, inertial_frame, hip_frame)
#hip_center.vel(inertial_frame)

hip_mass_center.v2pt_theory(hip_center, inertial_frame, hip_frame)
#hip_mass_center.vel(inertial_frame)

hip_right.v2pt_theory(hip_center, inertial_frame, hip_frame)
#hip_right.vel(inertial_frame)

knee_right.v2pt_theory(hip_right, inertial_frame, upper_leg_right_frame)
#knee_right.vel(inertial_frame)

ankle_right.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
#ankle_right.vel(inertial_frame)

lower_leg_left_mass_center.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
#lower_leg_left_mass_center.vel(inertial_frame)

lower_leg_right_mass_center.v2pt_theory(ankle_right, inertial_frame, lower_leg_right_frame)
#lower_leg_right_mass_center.vel(inertial_frame)

upper_leg_left_mass_center.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
#upper_leg_left_mass_center.vel(inertial_frame)

upper_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, upper_leg_right_frame)
upper_leg_right_mass_center.vel(inertial_frame)

l2*theta0'*R_2.x + l3*(-theta0' - theta1')*R_3.y + l2*(-theta0' - theta1' - theta2')/2*R_4.x

# Masses, Inertia, Rigid Bodies

In [19]:
lower_leg_mass, upper_leg_mass, hip_mass = symbols('m_L, m_U, m_H')

lower_leg_inertia, upper_leg_inertia, hip_inertia = symbols('I_Lz, I_Uz, I_Hz')

lower_leg_left_inertia_dyadic = inertia(lower_leg_left_frame, lower_leg_inertia, lower_leg_inertia, lower_leg_inertia)
lower_leg_left_central_inertia = (lower_leg_left_inertia_dyadic, lower_leg_left_mass_center)
lower_leg_left_inertia_dyadic.to_matrix(lower_leg_left_frame)

upper_leg_left_inertia_dyadic = inertia(upper_leg_left_frame, upper_leg_inertia, upper_leg_inertia, upper_leg_inertia)
upper_leg_left_central_inertia = (upper_leg_left_inertia_dyadic, upper_leg_left_mass_center)
upper_leg_left_inertia_dyadic.to_matrix(upper_leg_left_frame)

hip_inertia_dyadic = inertia(hip_frame, hip_inertia, hip_inertia, hip_inertia)
hip_central_inertia = (hip_inertia_dyadic, hip_mass_center)
hip_inertia_dyadic.to_matrix(hip_frame)

upper_leg_right_inertia_dyadic = inertia(upper_leg_right_frame, upper_leg_inertia, upper_leg_inertia, upper_leg_inertia)
upper_leg_right_central_inertia = (upper_leg_right_inertia_dyadic, upper_leg_right_mass_center)
upper_leg_right_inertia_dyadic.to_matrix(upper_leg_right_frame)

lower_leg_right_inertia_dyadic = inertia(lower_leg_right_frame, lower_leg_inertia, lower_leg_inertia, lower_leg_inertia)
lower_leg_right_central_inertia = (lower_leg_right_inertia_dyadic, lower_leg_right_mass_center)
lower_leg_right_inertia_dyadic.to_matrix(lower_leg_right_frame)

lower_leg_left = RigidBody('Lower Leg Left', lower_leg_left_mass_center, lower_leg_left_frame, lower_leg_mass, lower_leg_left_central_inertia)
                      
upper_leg_left = RigidBody('Upper Leg Left', upper_leg_left_mass_center, upper_leg_left_frame, upper_leg_mass, upper_leg_left_central_inertia)

hip = RigidBody('Hip', hip_mass_center, hip_frame, hip_mass, hip_central_inertia)

upper_leg_right = RigidBody('Upper Leg Right', upper_leg_right_mass_center, upper_leg_right_frame, upper_leg_mass, upper_leg_right_central_inertia)

lower_leg_right = RigidBody('Lower Leg Right', lower_leg_right_mass_center, lower_leg_right_frame, lower_leg_mass, lower_leg_right_central_inertia)


particles = []
particles.append(Particle('ankle_left', ankle_left, 0))
particles.append(Particle('knee_left', knee_left, 0))
particles.append(Particle('hip_left', hip_left, 0))
particles.append(Particle('hip_center', hip_center, 0))
particles.append(Particle('hip_right', hip_right, 0))
particles.append(Particle('knee_right', knee_right, 0))
particles.append(Particle('ankle_right', ankle_right, 0))
particles

mass_centers = []
mass_centers.append(Particle('lower_leg_left_mass_center', lower_leg_left_mass_center, lower_leg_mass))
mass_centers.append(Particle('upper_leg_left_mass_center', upper_leg_left_mass_center, upper_leg_mass))
mass_centers.append(Particle('hip_mass_center', hip_mass_center, hip_mass))
mass_centers.append(Particle('hip_mass_center', hip_mass_center, hip_mass))
mass_centers.append(Particle('hip_mass_center', hip_mass_center, hip_mass))
mass_centers.append(Particle('lower_leg_left_mass_center', lower_leg_right_mass_center, lower_leg_mass))
mass_centers.append(Particle('upper_leg_left_mass_center', upper_leg_right_mass_center, upper_leg_mass))
mass_centers

[lower_leg_left_mass_center,
 upper_leg_left_mass_center,
 hip_mass_center,
 hip_mass_center,
 hip_mass_center,
 lower_leg_left_mass_center,
 upper_leg_left_mass_center]

# Forces and Torques

In [20]:
g = symbols('g')

In [21]:
lower_leg_left_grav_force = (lower_leg_left_mass_center, -lower_leg_mass * g * inertial_frame.y)
upper_leg_left_grav_force = (upper_leg_left_mass_center, -upper_leg_mass * g * inertial_frame.y)
hip_grav_force = (hip_mass_center, -hip_mass * g * inertial_frame.y)
upper_leg_right_grav_force = (upper_leg_right_mass_center, -upper_leg_mass * g * inertial_frame.y)
lower_leg_right_grav_force = (lower_leg_right_mass_center, -lower_leg_mass * g * inertial_frame.y)

ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1 = dynamicsymbols('T_a0, T_k0, T_h0, T_h1, T_k1, T_a1')

lower_leg_left_torque_vector = ankle_torque0 * inertial_frame.z - knee_torque0 * inertial_frame.z
upper_leg_left_torque_vector = knee_torque0 * inertial_frame.z - hip_torque0 * inertial_frame.z
hip_left_torque_vector = hip_torque0 * inertial_frame.z - hip_torque1 * inertial_frame.z
hip_right_torque_vector = hip_torque1 * inertial_frame.z - knee_torque1 * inertial_frame.z
upper_leg_right_torque_vector = knee_torque1 * inertial_frame.z - ankle_torque1 * inertial_frame.z
lower_leg_right_torque_vector = ankle_torque1 * inertial_frame.z

lower_leg_left_torque = (lower_leg_left_frame, lower_leg_left_torque_vector)
upper_leg_left_torque = (upper_leg_left_frame, upper_leg_left_torque_vector)
hip_left_torque = (hip_frame, hip_left_torque_vector)
hip_right_torque = (hip_frame, hip_right_torque_vector)
upper_leg_right_torque = (upper_leg_right_frame, upper_leg_right_torque_vector)
lower_leg_right_torque = (lower_leg_right_frame, lower_leg_right_torque_vector)

# Equations of Motion

In [22]:
coordinates = [theta0, theta1, theta2, theta3, phi]
coordinates

[theta0, theta1, theta2, theta3, phi]

In [23]:
speeds = [omega0, omega1, omega2, omega3, psi]
speeds

[omega0, omega1, omega2, omega3, psi]

In [24]:
kinematical_differential_equations

[omega0 - theta0', omega1 - theta1', omega2 - theta2', omega3 - theta3', psi - phi']

In [25]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

In [26]:
loads = [lower_leg_left_grav_force,
         upper_leg_left_grav_force,
         hip_grav_force, 
         upper_leg_right_grav_force,
         lower_leg_right_grav_force,
         lower_leg_left_torque,
         upper_leg_left_torque,
         hip_left_torque,
         hip_right_torque,
         upper_leg_right_torque,
         lower_leg_right_torque]
loads

[(L_COMleft, - g*m_L*I.y),
 (U_COMleft, - g*m_U*I.y),
 (H_COMleft, - g*m_H*I.y),
 (U_COMright, - g*m_U*I.y),
 (L_COMright, - g*m_L*I.y),
 (R_1, (T_a0 - T_k0)*I.z),
 (R_2, (-T_h0 + T_k0)*I.z),
 (R_3, (T_h0 - T_h1)*I.z),
 (R_3, (T_h1 - T_k1)*I.z),
 (R_4, (-T_a1 + T_k1)*I.z),
 (R_5, T_a1*I.z)]

In [27]:
bodies = [lower_leg_left, upper_leg_left, hip, upper_leg_right, lower_leg_right]
bodies

[Lower Leg Left, Upper Leg Left, Hip, Upper Leg Right, Lower Leg Right]

In [28]:
fr, frstar = kane.kanes_equations(loads, bodies)

In [29]:
trigsimp(fr + frstar)

Matrix([
[-g*l1*m_L*sin(-phi + theta0 + theta1 + theta2 + theta3)/2 - g*l2*m_H*sin(phi - theta0) - g*l2*m_L*sin(phi - theta0) - g*l2*m_L*sin(-phi + theta0 + theta1 + theta2) - 3*g*l2*m_U*sin(phi - theta0)/2 - g*l2*m_U*sin(-phi + theta0 + theta1 + theta2)/2 + g*l3*m_H*cos(-phi + theta0 + theta1)/2 + g*l3*m_L*cos(-phi + theta0 + theta1) + g*l3*m_U*cos(-phi + theta0 + theta1) - l1*l2*m_L*(-omega0 - omega1 - omega2)**2*sin(theta3)/2 - l1*l2*m_L*(-omega0 - omega1 - omega2 - omega3)**2*sin(theta1 + theta2 + theta3)/2 + l1*l2*m_L*(-omega0 - omega1 - omega2 - omega3)**2*sin(theta3)/2 + l1*l2*m_L*omega0**2*sin(theta1 + theta2 + theta3)/2 - l1*l3*m_L*(-omega0 - omega1)**2*cos(theta2 + theta3)/2 + l1*l3*m_L*(-omega0 - omega1 - omega2 - omega3)**2*cos(theta2 + theta3)/2 - l2**2*m_L*(-omega0 - omega1 - omega2)**2*sin(theta1 + theta2) + l2**2*m_L*omega0**2*sin(theta1 + theta2) - l2**2*m_U*(-omega0 - omega1 - omega2)**2*sin(theta1 + theta2)/2 + l2**2*m_U*omega0**2*sin(theta1 + theta2)/2 + l2*l3*m_H*(

Keep in mind that out utlimate goal is to have the equations of motion in first order form:

$$ \dot{\mathbf{x}} = \mathbf{g}(\mathbf{x}, t) $$

The equations of motion are linear in terms of the derivatives of the generalized speeds and the `KanesMethod` class automatically puts the equations in a more useful form for the next step of numerical simulation:

$$ \mathbf{M}(\mathbf{x}, t)\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, t) $$

Note that

$$ \mathbf{g} = \mathbf{M}^{-1}(\mathbf{x}, t) \mathbf{f}(\mathbf{x}, t) $$

and that $\mathbf{g}$ can be computed analytically but for non-toy problems, it is best to do this numerically. So we will simply generate the $\mathbf{M}$ and $\mathbf{f}$ matrices for later use.

The mass matrix, $\mathbf{M}$, can be accessed with the `mass_matrix` method (use `mass_matrix_full` to include the kinematical differential equations too. We can use `trigsimp` again to make this relatively compact: 

In [30]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

Matrix([
[1, 0, 0, 0, 0,                                                                                                                                                                                                                                                                                                                                                                                    0,                                                                                                                                                                                                                                                                                                                                                      0,                                                                                                                                                                                                                                               0,                 

The right hand side, $\mathbf{f}$, is a vector function of all the non-inertial forces (gyroscopic, external, coriolis, etc):

In [31]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# Simulation

In [39]:
from scipy.integrate import odeint
from sympy.utilities.codegen import codegen
from pydy.codegen.ode_function_generators import generate_ode_function
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
rcParams['figure.figsize'] = (14.0, 6.0)

specified = [ankle_torque0, knee_torque0, hip_torque0, hip_torque1, knee_torque1, ankle_torque1]
numerical_specified = zeros(6)

x0 = zeros(10)
x0

x0[0] = deg2rad(80)
x0[1] = -deg2rad(80)
x0[2] = -deg2rad(80)
x0[3] = deg2rad(80)
x0[4] = 0

x0

array([ 1.3962634, -1.3962634, -1.3962634,  1.3962634,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ])

In [33]:
#%% Jacobian for the right ankle, which we will try to leave where it is when
#   moving the hip center
print ('calculating jacobian for the right ankle')
F0 = ankle_right.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
F0 = Matrix([F0[0], F0[1]])
F0

calculating jacobian for the right ankle


Matrix([
[-l1*sin(-phi + theta0 + theta1 + theta2 + theta3) - l1*sin(phi) - l2*sin(phi - theta0) - l2*sin(-phi + theta0 + theta1 + theta2) + l3*cos(-phi + theta0 + theta1)],
[-l1*cos(-phi + theta0 + theta1 + theta2 + theta3) + l1*cos(phi) + l2*cos(phi - theta0) - l2*cos(-phi + theta0 + theta1 + theta2) - l3*sin(-phi + theta0 + theta1)]])

In [34]:
J_ankleRight = F0.jacobian([theta0, theta1, theta2, theta3, phi])
J_ankleRight

Matrix([
[-l1*cos(-phi + theta0 + theta1 + theta2 + theta3) + l2*cos(phi - theta0) - l2*cos(-phi + theta0 + theta1 + theta2) - l3*sin(-phi + theta0 + theta1), -l1*cos(-phi + theta0 + theta1 + theta2 + theta3) - l2*cos(-phi + theta0 + theta1 + theta2) - l3*sin(-phi + theta0 + theta1), -l1*cos(-phi + theta0 + theta1 + theta2 + theta3) - l2*cos(-phi + theta0 + theta1 + theta2), -l1*cos(-phi + theta0 + theta1 + theta2 + theta3),  l1*cos(-phi + theta0 + theta1 + theta2 + theta3) - l1*cos(phi) - l2*cos(phi - theta0) + l2*cos(-phi + theta0 + theta1 + theta2) + l3*sin(-phi + theta0 + theta1)],
[ l1*sin(-phi + theta0 + theta1 + theta2 + theta3) + l2*sin(phi - theta0) + l2*sin(-phi + theta0 + theta1 + theta2) - l3*cos(-phi + theta0 + theta1),  l1*sin(-phi + theta0 + theta1 + theta2 + theta3) + l2*sin(-phi + theta0 + theta1 + theta2) - l3*cos(-phi + theta0 + theta1),  l1*sin(-phi + theta0 + theta1 + theta2 + theta3) + l2*sin(-phi + theta0 + theta1 + theta2),  l1*sin(-phi + theta0 + theta1 + theta

In [35]:
F1 = hip_center.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
F1 = Matrix([F1[0], F1[1]])
F1

Matrix([
[-l1*sin(phi) - l2*sin(phi - theta0) + l3*cos(-phi + theta0 + theta1)/2],
[ l1*cos(phi) + l2*cos(phi - theta0) - l3*sin(-phi + theta0 + theta1)/2]])

In [36]:
J_hipCenter = F1.jacobian([theta0, theta1, theta2, theta3, phi])
J_hipCenter

Matrix([
[l2*cos(phi - theta0) - l3*sin(-phi + theta0 + theta1)/2, -l3*sin(-phi + theta0 + theta1)/2, 0, 0, -l1*cos(phi) - l2*cos(phi - theta0) + l3*sin(-phi + theta0 + theta1)/2],
[l2*sin(phi - theta0) - l3*cos(-phi + theta0 + theta1)/2, -l3*cos(-phi + theta0 + theta1)/2, 0, 0, -l1*sin(phi) - l2*sin(phi - theta0) + l3*cos(-phi + theta0 + theta1)/2]])

In [37]:
#%% we stack the two jacobians
J = J_ankleRight.col_join(J_hipCenter)
J.shape

(4, 5)

In [40]:
#%% lets try the pseudo inverse with a couple of real values
values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, theta0: x0[0], theta1: x0[1], theta2: x0[2], theta3: x0[3], phi: x0[4]}
Jpinv = J.subs(values).evalf().pinv()
Jpinv

Matrix([
[ -0.0459332479170267,  0.219478074655015,   -2.3994104929455,  -2.31969900118024],
[   0.235092856483776,  -1.12331981399207, -0.514831317577474,   1.87254742697749],
[  -0.235092856483778, -0.757099837648197,  0.514831317577473,   1.88829187630304],
[   -2.45406675208297,   2.10175902875642,   2.39941049294551,  -2.32277520564262],
[-0.00872301122043817, 0.0416802598264825,  -2.48089742314152, -0.440525356532326]])

In [41]:
#%% we stack the two endeffektor points, which we will evaluate in the integration
F2 = F0.col_join(F1)
F2

Matrix([
[-l1*sin(-phi + theta0 + theta1 + theta2 + theta3) - l1*sin(phi) - l2*sin(phi - theta0) - l2*sin(-phi + theta0 + theta1 + theta2) + l3*cos(-phi + theta0 + theta1)],
[-l1*cos(-phi + theta0 + theta1 + theta2 + theta3) + l1*cos(phi) + l2*cos(phi - theta0) - l2*cos(-phi + theta0 + theta1 + theta2) - l3*sin(-phi + theta0 + theta1)],
[                                                                                           -l1*sin(phi) - l2*sin(phi - theta0) + l3*cos(-phi + theta0 + theta1)/2],
[                                                                                            l1*cos(phi) + l2*cos(phi - theta0) - l3*sin(-phi + theta0 + theta1)/2]])

In [42]:
#%% this defines how long you want to simulate, we simulate for one second with 30 fps
#   simulation can take quite a while...
frames_per_sec = 30
final_time = 1

t = linspace(0.0, final_time, final_time * frames_per_sec)

In [43]:
numerical_constants = array([0.42,  # lower_leg_length [m]
                             0.54, # upper_leg_length [m]
                             0.2, # hip_length
                                                          
                             1.0,  # lower_leg_mass [kg]
                             1.5,  # upper_leg_mass [kg]
                             2.0,  # hip_mass [kg]
                             
                             0.1,  # lower_leg_inertia [kg*m^2]                             
                             0.2,  # upper_leg_inertia [kg*m^2]
                             0.1,  # hip_inertia [kg*m^2]
                             
                             9.81],  # acceleration due to gravity [m/s^2]
                            ) 
numerical_constants

array([ 0.42,  0.54,  0.2 ,  1.  ,  1.5 ,  2.  ,  0.1 ,  0.2 ,  0.1 ,  9.81])

In [44]:
constants = [lower_leg_length,
             upper_leg_length,
             hip_length,
             
             lower_leg_mass,
             upper_leg_mass,
             hip_mass,
             
             lower_leg_inertia,
             upper_leg_inertia,
             hip_inertia,
             
             g]
constants

[l1, l2, l3, m_L, m_U, m_H, I_Lz, I_Uz, I_Hz, g]

In [45]:
#%%
x0 = zeros(10)
x0

x0[0] = deg2rad(80)
x0[1] = -deg2rad(80)
x0[2] = -deg2rad(80)
x0[3] = deg2rad(80)
x0[4] = 0

x0

array([ 1.3962634, -1.3962634, -1.3962634,  1.3962634,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ])

In [46]:

#%% Use this for full kinematics simulation
#right_hand_side = generate_ode_function(forcing_vector, coordinates, speeds, 
#                                        constants, mass_matrix=mass_matrix, 
#                                        specifieds=specified, generator='cython')
#
#args = {'constants': numerical_constants,
#        'specified': numerical_specified}
#                                        
#right_hand_side(x0, 0.0, numerical_specified, numerical_constants)        
##%%
#y = odeint(right_hand_side, x0, t, args=(numerical_specified, numerical_constants))
#y


#%% only useful when you want to simulate full kinematics, use in right_hand_side equation
#from sympy import lambdify, solve
#M_func = lambdify(coordinates + speeds + constants + specified, mass_matrix) # Create a callable function to evaluate the mass matrix 
#f_func = lambdify(coordinates + speeds + constants + specified, forcing_vector)     # Create a callable function to evaluate the forcing vector       
#%% inverse kinematics gains, you can balance here how much the respective 
#   targets should be tried to reach
kp = np.eye(4)
kp[0,0] = 10
kp[1,1] = 10
kp[2,2] = 10
kp[3,3] = 10
kp
#%%
kpN = np.zeros((4,4),dtype=float)
kpN[1,1] = 10
kpN
#%% These are our target points
L = 1 # this defines how far apart the left and right ankle should be, y=0 so the 
      # foot does not lift of the ground
      # the last two values are x and y coordinates for the hip center
x_des = Matrix([L,0,0.7,0.1])
x_des.shape
#%% This is the intergration function (the inverse kinematics)
#   I highly recommend you have a look at this textbook http://smpp.northwestern.edu/savedLiterature/Spong_Textbook.pdf
#   especially chapter 5.10 Inverse Velocity and Acceleration
i = 0
def right_hand_side(x, t, args):
    """Returns the derivatives of the states.

    Parameters
    ----------
    x : ndarray, shape(2 * (n + 1))
        The current state vector.
    t : float
        The current time.
    args : ndarray
        The constants.

    Returns
    -------
    dx : ndarray, shape(2 * (n + 1))
        The derivative of the state.
    
    """
    global i
    r = 0.0                                  # The input force is always zero     
    arguments = np.hstack((x, args))      # States, input, and parameters
    values = {lower_leg_length: 0.4, upper_leg_length: 0.54, hip_length: 0.2, 
              theta0: x[0], theta1: x[1], theta2: x[2], theta3: x[3], phi: x[4],
                omega0: 0, omega1: 0, omega2: 0, omega3: 0, psi: 0}
    Jpinv = J.subs(values).evalf().pinv()
    # use this for nullspace movements
#    N=np.eye(4)-Jpinv*J.subs(values).evalf()
    x_current = F2.subs(values).evalf()                                         # use this for nullspace movements
    dq = np.array(Jpinv*(kp*( x_des - x_current ))).astype(float).T[0]# + N*(kpN*(Matrix([0,-deg2rad(80),0,0,0])-Matrix([x[0],x[1],x[2],x[3])))).astype(float).T[0]
    dq = np.hstack((dq,np.zeros(5,dtype=float)))
    if i%10==0:
        print(x_current)
    i = i+1
    return dq
    # use this for full kinematic simulation
#    dq = np.array(np.linalg.solve(M_func(*arguments),  # Solving for the derivatives
#                  f_func(*arguments))).T[0]
#    return dq

In [47]:
#%% Ok lets simulate
print('integrating')
args = (np.hstack((numerical_constants,)),)
y = odeint(right_hand_side, x0, t, args)#,full_output = 1,mxstep=1000000
#y = np.hstack((y,np.zeros(y.shape)))
print(y)
print('done integrating')

integrating
Matrix([[1.26359237325318], [0], [0.631796186626592], [0.493770015940142]])
Matrix([[1.26175159570018], [6.37010906447567e-9], [0.632272477736748], [0.491020164371748]])
Matrix([[1.25528230436820], [-8.25783376997690e-9], [0.633946400654344], [0.481355924543045]])
Matrix([[1.24204513938050], [-4.32124976079695e-9], [0.637371479067839], [0.461581463804779]])
Matrix([[1.22016985269966], [-7.65121452371391e-9], [0.643031651745630], [0.428902850571606]])
Matrix([[1.19347469825923], [-4.40686484179387e-9], [0.649938948994721], [0.389024040678037]])
Matrix([[1.16441873966249], [-1.32369283719833e-9], [0.657457097886850], [0.345618520687556]])
Matrix([[1.13086159526863], [3.19119596654989e-9], [0.666139915607050], [0.295488862780702]])
Matrix([[1.09762332427629], [8.60721838030418e-9], [0.674740225069592], [0.245835558050327]])
Matrix([[1.06845152046108], [1.41858201431304e-8], [0.682288346687779], [0.202256984915244]])
Matrix([[1.04481151791494], [1.23655330506317e-8], [0.6884051

In [48]:
#%% Plot 
# here we plot a little
print("Plotting")
plot(t, rad2deg(y[:, :5]))
xlabel('Time [s]')
ylabel('Angle [deg]')
legend(["${}$".format(vlatex(c)) for c in coordinates])

Plotting


# Visualization

In [49]:
from pydy.viz.shapes import Cylinder, Sphere
from pydy.viz.scene import Scene
from pydy.viz.visualization_frame import VisualizationFrame

# stack the lengths and use the bodies from kinematics
lengths = [lower_leg_length, upper_leg_length, hip_length, hip_length, hip_length, upper_leg_length, lower_leg_length]
bodies = [lower_leg_left, upper_leg_left, hip, hip, hip, upper_leg_right, lower_leg_right]

viz_frames = []
colors = ['yellow','green','red','red','red','green','blue']

for i, (body, particle, mass_center) in enumerate(zip(bodies, particles, mass_centers)):
#        body_shape = Cylinder(name='cylinder{}'.format(i),
#                              radius=0.05,
#                              length=lengths[i],
#                              color='red')
#    
#        viz_frames.append(VisualizationFrame('link_frame{}'.format(i), body,
#                                             body_shape))
            
    particle_shape = Sphere(name='sphere{}'.format(i),
                            radius=0.06,
                            color=colors[i])
                            
    
    viz_frames.append(VisualizationFrame('particle_frame{}'.format(i),
                                         body.frame,
                                         particle,
                                         particle_shape))
                                         
    mass_center_shape = Sphere(name='sphere{}'.format(i),
                            radius=0.02,
                            color='black')
                                         
    viz_frames.append(VisualizationFrame('mass_center_frame{}'.format(i),
                                         body.frame,
                                         mass_center,
                                         mass_center_shape))
                                             
target_shape = Sphere(name='sphere{}'.format(i),
                            radius=0.02,
                            color='green')    

target_right_leg = Point('target_right_leg')
target_right_leg.set_pos(origin, (x_des[2] * inertial_frame.x)+(x_des[3] * inertial_frame.y))                              

                                     
viz_frames.append(VisualizationFrame('target_frame_right',
                                     inertial_frame,
                                     target_right_leg,
                                     target_shape))

## Now the visualization frames can be passed in to create a scene.
scene = Scene(inertial_frame, origin, *viz_frames)

# Provide the data to compute the trajectories of the visualization frames.
scene.constants = dict(zip(constants, numerical_constants))
scene.states_symbols = coordinates+speeds
scene.states_trajectories = y

scene.display()

/home/letrend/workspace/roboy-ros-control/python/pydy-resources
('Serving HTTP on', '127.0.0.1', 'port', 8001, '...')
To view visualization, open:

http://localhost:8001/index.html?load=2017-06-28_02-51-33_scene_desc.json
Press Ctrl+C to stop server...


# export to c header

In [50]:
t = symbols('t')
a0 = ankle_left.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
k0 = knee_left.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
hl = hip_left.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
hc = hip_center.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
hr = hip_right.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
k1 = knee_right.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
a1 = ankle_right.pos_from(origin).express(inertial_frame).simplify().to_matrix(inertial_frame)
[(c_name, c_code), (h_name, c_header)] = codegen( 
    [("Jacobian",J), 
     ("ankle_right_hip_center",F2), 
     ("ankle_left", a0),
     ("knee_left", k0),
     ("hip_left", hl),
     ("hip_center", hc),
     ("hip_right", hr),
     ("knee_right", k1),
     ("ankle_right",a1)
    ] ,
    "C", "PaBiRoboy_DanceControl", project='PaBiRoboy_DanceControl',global_vars=(t,lower_leg_length, upper_leg_length, hip_length, theta0,theta1,theta2,theta3), 
    header=True, empty=False)
print(c_code)

127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /index.html?load=2017-06-28_02-51-33_scene_desc.json HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /css/slider.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /css/codemirror/codemirror.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /css/codemirror/blackboard.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /js/external/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /js/external/jquery/jquery-ui.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /js/external/bootstrap/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /js/external/codemirror/codemirror.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2017 02:51:40] "GET /js/external/codemirror/javascript-mode.js HTTP/1.1" 200 -
127.0.

/******************************************************************************
 *                       Code generated with sympy 1.0                        *
 *                                                                            *
 *              See http://www.sympy.org/ for more information.               *
 *                                                                            *
 *               This file is part of 'PaBiRoboy_DanceControl'                *
 ******************************************************************************/
#include "PaBiRoboy_DanceControl.h"
#include <math.h>
void Jacobian(double *out_7040956801295068946) {
   out_7040956801295068946[0] = -l1*cos(-phi(t) + theta0(t) + theta1(t) + theta2(t) + theta3(t)) + l2*cos(phi(t) - theta0(t)) - l2*cos(-phi(t) + theta0(t) + theta1(t) + theta2(t)) - l3*sin(-phi(t) + theta0(t) + theta1(t));
   out_7040956801295068946[1] = -l1*cos(-phi(t) + theta0(t) + theta1(t) + theta2(t) + theta3(t)) - l2*cos(-phi(t)